In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/pSp_self_customize')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import dlib
import glob

import cv2
import numpy as np


In [3]:
test_image = "image_datasets/inversion_images/barak_obama.jpg"

"""
input_path = "image_datasets/inversion_images"
original_output_path = "image_datasets/blurred_ori_inversion_images"
processed_output_path = "image_datasets/blurred_inversion_images"
"""
input_path = "image_datasets/ffhq0k_5k"
original_output_path = "image_datasets/blurred_ori_images"
processed_output_path = "image_datasets/blurred_pro_images"

try:
  os.makedirs(original_output_path)
  os.makedirs(processed_output_path)
except:
  pass


In [4]:
#Load face landmark detection
detector = dlib.get_frontal_face_detector()


In [5]:
def blur(img, face_rectangle, ksize, sigma):
  topleft_x = face_rectangle.left()
  topleft_y = face_rectangle.top()
  w = face_rectangle.right() - face_rectangle.left()
  h = face_rectangle.bottom() - face_rectangle.top()

  roi = img[topleft_y: topleft_y+h, topleft_x: topleft_x+w]
  blur = cv2.GaussianBlur(roi, (ksize,ksize), sigmaX=sigma) #  sigmaY default=0 if sigmaY is zero, it is set to be equal to sigmaX.

  img[topleft_y: topleft_y+h, topleft_x: topleft_x+w] = blur

  return img


In [6]:
counts =0

for f in glob.glob(os.path.join(input_path, "*.png")):
  #print("Processing file: {}".format(f))
  image_np = dlib.load_rgb_image(f)
  dets = detector(image_np,1)

  image_cv = cv2.imread(f)
  try:
    blurred_image = blur(img=image_cv.copy(), face_rectangle=dets[0], ksize=101, sigma=0)
    f_name = f.split("/")[-1]
    cv2.imwrite(os.path.join(original_output_path,f_name), image_cv)
    cv2.imwrite(os.path.join(processed_output_path,"blurred_"+f_name), blurred_image)
  except:
    continue

  counts += 1
  if counts % 10 == 0:
        print('Already processing {} files\r'.format(counts), end='', flush=True)

print("Total {} images have been mosaiced!".format(counts))


Total 13 images have been mosaiced!


In [7]:
counts

13